In [1]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [2]:
import warnings
warnings.simplefilter('ignore') #ignores simple warning

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as tk
import matplotlib.patches as mpatches
import matplotlib as mpl

In [4]:
#Read data
datav10 = xr.open_mfdataset('v10\*.nc',combine = 'by_coords')
datav925 = xr.open_mfdataset('v925\*.nc',combine = 'by_coords')
datau925 = xr.open_mfdataset('u925\*.nc',combine = 'by_coords')
datamslp = xr.open_mfdataset('mslp\*.nc',combine = 'by_coords')/100

In [5]:
#CS no-CS Classification
#u<=0
ucs=datau925['u'].sel(lat= slice(5,10), lon=slice(107,115), time=slice('1981-11-01T00:00:00.000000000','2021-03-31T00:00:00.000000000'))
ucsidx=ucs.rolling(time=3, center=True).mean()
ucsidx=ucsidx.mean(dim=('lat', 'lon'))

#v<0
vcs=datav925['v'].sel(lat= slice(5,10), lon=slice(107,115), time=slice('1981-11-01T00:00:00.000000000','2021-03-31T00:00:00.000000000'))
vcsidx=vcs.rolling(time=3, center=True).mean()
vcsidx=vcsidx.mean(dim=('lat', 'lon'))

#ws
wsidx=(vcsidx**2+ucsidx**2)**0.5
wsdjf=wsidx.sel(time=wsidx.time.dt.month.isin([1, 2, 12]))
thws=(np.mean(wsdjf)+0.5*np.std(wsdjf))
print('threshold CS wind speed idx= ', thws.values, 'm/s')

wdcsidx=((vcsidx<0)*(ucsidx<=0)*(wsidx>thws))

#MSLP criteria (18-22 LU, 105-122 BT)
mslcs=datamslp.sel(lat= slice(18,22), lon=slice(105,122), time=slice('1981-11-01T00:00:00.000000000','2021-03-31T00:00:00.000000000'))
mslcs=mslcs.rolling(time=3, center=True).mean()
mslcs=mslcs.sel(time=mslcs.time.dt.month.isin([1, 2, 3, 11, 12]))['msl']
mslcs=mslcs.mean(dim=('lat', 'lon'))
mslcsdjf=mslcs.sel(time=mslcs.time.dt.month.isin([1, 2, 12]))
thmsl=(np.mean(mslcsdjf)+0.25*np.std(mslcsdjf))
print('threshold CS mslp idx= ',thmsl.values, 'hPa')
mslcsidx=mslcs>thmsl

threshold CS wind speed idx=  11.377695 m/s
threshold CS mslp idx=  1017.97516 hPa


In [6]:
wdcsidxdjf=wdcsidx.sel(time=wdcsidx.time.dt.month.isin([1, 2, 12]))
tgl=wdcsidxdjf['time']
tglmin1=tgl-np.timedelta64(1, 'D')
mslpreq=np.logical_or(mslcsidx.sel(time=tgl).values,mslcsidx.sel(time=tglmin1).values)
cswmslpevt=wdcsidxdjf.sel(time=tgl)*mslpreq

In [7]:
cswmslpevt = xr.Dataset(
     {"index": (("time"), cswmslpevt.values)},
    coords={
    "time": cswmslpevt['time'].values,
    },
    )
cswmslpevt=cswmslpevt['index']

In [8]:
censidx=datav10.sel(lat= slice(-5,0), lon=slice(105,110), time=slice('1981-11-01T00:00:00.000000000','2021-03-31T00:00:00.000000000'))
censidx=censidx.rolling(time=3, center=True).mean()
censidx=censidx.sel(time=censidx.time.dt.month.isin([1, 2, 3, 11, 12]))['v10']
censidx=censidx.mean(dim=('lat', 'lon'))
censidx=censidx.load()
censidxdjf=censidx.sel(time=censidx.time.dt.month.isin([1, 2, 12]))
thcens=(np.mean(censidxdjf)-1*np.std(censidxdjf))
censevt=censidx<thcens
print('threshold CENS wind idx= ', thcens.values, 'm/s')

threshold CENS wind idx=  -4.168241858482361 m/s


In [9]:
ucsidx=ucsidx.load()
vcsidx=vcsidx.load()
wsidx=wsidx.load()
censidx=censidx.load()

In [11]:
#Peak CS from V component
start=[]
duration=[]
end=[]
peak=[]
wsmag=[]
vmag=[]
umag=[]
censid=[]
maxcens=[]
#peakcens=[]
for year in tqdm(np.arange(1981,2021,1)):
    csslice=cswmslpevt.sel(time=pd.date_range(start=str(year)+'-12-01', end=str(year+1)+'-02-28'))
    tgl=csslice[csslice==1]['time']
    if len(tgl)==0:
        continue
    tgl=np.append(tgl, tgl[-1]+np.timedelta64(2, 'D'))
    dur=1
    evt=0
    for i in range(len(tgl)-1):
        lagday=(tgl[i+1]-tgl[i])/np.timedelta64(1, 'D')
        if lagday==1:
            dur+= 1
        else:
            evt+= 1
            duration.append(dur)
            end.append(tgl[i])
            if i==0:
                start.append(tgl[i])
            else:
                start.append((tgl[i]-np.timedelta64(int(dur)-1, 'D')))
            csidxevt=vcsidx.sel(time=slice(tgl[i]-np.timedelta64(int(dur)-1, 'D'),tgl[i]))
            maxval=csidxevt.where(csidxevt==csidxevt.min(), drop=True).squeeze()
            vmag.append(maxval.values)
            peak.append(maxval['time'].values)
            wsmag.append(wsidx.sel(time=maxval['time'].values).values)
            umag.append(ucsidx.sel(time=maxval['time'].values).values)
            if censevt.sel(time=slice(maxval['time'].values,maxval['time'].values+np.timedelta64(1, 'D'))).max()==1:
                censid.append(1)
            else:
                censid.append(0)
            censidxevt=censidx.sel(time=slice(maxval['time'].values,maxval['time'].values+np.timedelta64(1, 'D')))
            maxvalcens=censidxevt.where(censidxevt==censidxevt.min(), drop=True).squeeze()
            maxcens.append(maxvalcens.values)
            #peakcens.append(maxvalcens['time'].values)
            dur=1

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 12.27it/s]


In [12]:
tglcstab=pd.DataFrame({'duration'    : duration,
                        'time_start' : start,
                        'time_end'   : end,
                        'time_peak'  : peak,
                        'vmagcs'       : vmag,
                        'umagcs'       : umag,
                        'wsmagcs'     : wsmag,
                        'censevt'    : censid,
                        'censidx'    : maxcens,
                        #'peakcens'   : peakcens
                      })

print(tglcstab.to_string())

     duration time_start   time_end  time_peak       vmagcs       umagcs     wsmagcs  censevt      censidx
0           3 1981-12-04 1981-12-06 1981-12-05   -10.300501   -7.4688506   12.723366        0   -3.2029169
1          11 1981-12-12 1981-12-22 1981-12-19    -11.35001    -9.791237   14.989697        1    -4.265283
2           3 1982-01-06 1982-01-08 1982-01-07   -11.147718   -7.5051866   13.438729        0   -3.2280946
3           4 1982-01-17 1982-01-20 1982-01-18    -9.964183    -7.581951    12.52082        1     -7.36791
4           5 1982-01-27 1982-01-31 1982-01-27    -8.828562    -7.651181    11.68264        1   -4.5575414
5           3 1982-02-14 1982-02-16 1982-02-15    -9.188368   -9.4450445   13.177062        1    -4.601391
6           3 1982-12-13 1982-12-15 1982-12-15    -7.420511    -8.899022   11.586914        0  -0.88717526
7           6 1982-12-17 1982-12-22 1982-12-18   -10.474841    -7.614261   12.949876        0   -1.1425369
8           3 1982-12-26 1982-12-28 1

In [14]:
#tglcstab.to_excel('E:\. Disertasi S3 Bismillah\Olah Data\CS Lim Index V Based Event Data RM3.xlsx')